# Predictive Maintenance with Linear Regression Alerts

## Manufacturing Robot Current Monitoring - Failure Prediction System

This notebook implements a Predictive Maintenance Alert System using Linear Regression models to detect anomalies and predict failures in industrial robot current data.

### Workflow:
1. **Database Integration** - Connect to Neon.tech PostgreSQL, ingest training data
2. **Model Training** - Fit univariate linear regression for each axis (Time → Current)
3. **Residual Analysis** - Analyze prediction errors to discover thresholds
4. **Alert System** - Implement ALERT/ERROR detection based on sustained deviations
5. **Testing** - Process synthetic failure data and log events
6. **Visualization** - Generate alert dashboard

## Cell 1: Imports and Setup

In [ ]:
import sys
import os

# Add src directory to Python path
project_root = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == 'notebook' else os.getcwd()
src_dir = os.path.join(project_root, 'src')
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

print(f"Project root: {project_root}")
print(f"Source directory: {src_dir}")

# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Use Qt backend for separate windows
%matplotlib qt

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Import custom modules
from database_utils import (connect_to_db, create_training_table, 
                            ingest_training_data, query_training_data,
                            get_training_record_count, create_alerts_table,
                            clear_training_table)
from linear_regression_model import RobotRegressionModels
from alert_system import AlertSystem, AlertThresholds

print("\nLibraries imported successfully")
print(f"Matplotlib backend: {plt.get_backend()}")

## Cell 2: Database Integration

Connect to Neon.tech PostgreSQL and ingest training data.

In [ ]:
# Database configuration (Neon.tech PostgreSQL)
db_config = {
    'host': 'ep-polished-snow-ahx3qiod-pooler.c-3.us-east-1.aws.neon.tech',
    'database': 'neondb',
    'user': 'neondb_owner',
    'password': 'npg_JlIENr3i4AbL',
    'port': 5432,
    'sslmode': 'require'
}

# Training data path
training_csv_path = os.path.join(project_root, 'data', 'robots_combined_traindata.csv')

print("=" * 70)
print("DATABASE INTEGRATION")
print("=" * 70)

# Connect to database
conn = connect_to_db(db_config)

# Create tables
create_training_table(conn)
create_alerts_table(conn)

# Check existing records
existing_count = get_training_record_count(conn)
print(f"\nExisting training records in database: {existing_count:,}")

In [ ]:
# Ingest training data (only if table is empty or needs refresh)
FORCE_REINGEST = True  # Set to True to reload data after regenerating CSVs

if existing_count == 0 or FORCE_REINGEST:
    if FORCE_REINGEST and existing_count > 0:
        print("Clearing existing training data...")
        clear_training_table(conn)
    
    print(f"\nIngesting training data from: {training_csv_path}")
    records_ingested = ingest_training_data(conn, training_csv_path)
    print(f"Total records ingested: {records_ingested:,}")
else:
    print(f"Using existing {existing_count:,} records in database")
    print("(Set FORCE_REINGEST=True to reload data)")

In [ ]:
# Query training data from database
print("\nQuerying training data from database...")
df_training = query_training_data(conn, robot_name='Robot A')  # Use Robot A as baseline

print(f"\nTraining Data Shape: {df_training.shape}")
print(f"Columns: {list(df_training.columns)}")
print(f"\nFirst 5 rows:")
df_training.head()

## Cell 3: Model Training

Fit univariate linear regression for each axis:
- **Model**: y = slope × time_index + intercept
- **Axes**: axis_1 through axis_12 (all 12 axes)

In [ ]:
# Initialize and train regression models for all 12 axes
models = RobotRegressionModels()
model_params = models.train_all_axes(df_training)

# Display model summary
print("\n" + "=" * 70)
print("MODEL PARAMETERS SUMMARY (All 12 Axes)")
print("=" * 70)
model_summary = models.get_model_summary()
print(model_summary.to_string())

# Save model parameters
params_path = os.path.join(project_root, 'data', 'model_params.csv')
models.save_model_params(params_path)

## Cell 4: Regression Visualization

Scatter plots with regression lines for all 12 axes.

In [ ]:
# Plot regression lines for training data (all 12 axes)
print("Generating regression plots for all 12 axes...")
fig_regression = models.plot_regression_lines(
    df_training, 
    title_prefix="Training Data: ",
    save_path=os.path.join(project_root, 'alerts', 'regression_lines.png')
)
plt.show()

## Cell 5: Residual Analysis

Analyze residuals (observed - predicted) to:
1. Understand prediction error distribution
2. Identify outlier patterns
3. Determine appropriate thresholds for alerts

In [ ]:
# Plot residual distributions for all 12 axes
print("Generating residual analysis plots...")
fig_residuals = models.plot_residual_analysis(
    df_training,
    save_path=os.path.join(project_root, 'alerts', 'residual_histograms.png')
)
plt.show()

In [ ]:
# Residual boxplots for outlier identification
fig_boxplots = models.plot_residual_boxplots(
    df_training,
    save_path=os.path.join(project_root, 'alerts', 'residual_boxplots.png')
)
plt.show()

In [ ]:
# Compute detailed residual statistics for all 12 axes
print("=" * 70)
print("RESIDUAL STATISTICS (All 12 Axes)")
print("=" * 70)

time_index = np.arange(len(df_training))
residual_stats = []

for axis_name in models.AXIS_NAMES:
    if axis_name in df_training.columns and models.models[axis_name].is_fitted:
        residuals = models.models[axis_name].get_residuals(time_index, df_training[axis_name])
        residuals = residuals[~np.isnan(residuals)]
        
        if len(residuals) > 0:
            stats = {
                'axis': axis_name,
                'mean': np.mean(residuals),
                'std': np.std(residuals),
                'min': np.min(residuals),
                'max': np.max(residuals),
                'q1': np.percentile(residuals, 25),
                'median': np.median(residuals),
                'q3': np.percentile(residuals, 75),
                'p95': np.percentile(residuals, 95),
                'p99': np.percentile(residuals, 99)
            }
            residual_stats.append(stats)
        else:
            print(f"  {axis_name}: No valid data")
    else:
        print(f"  {axis_name}: No model fitted (no data)")

if residual_stats:
    df_residual_stats = pd.DataFrame(residual_stats).round(4)
    print(df_residual_stats.to_string())

## Cell 6: Threshold Discovery & Justification

Based on residual analysis, define alert thresholds:

- **MinC**: Minimum deviation for ALERT (scaled by residual_std)
- **MaxC**: Maximum deviation for ERROR (scaled by residual_std)
- **T**: Minimum sustained duration in seconds

### Justification:

1. **MinC = 2.0**: Values exceeding 2σ from the regression line represent approximately 5% of normal variance (beyond 95% confidence interval). In predictive maintenance, this indicates early signs of degradation.

2. **MaxC = 3.0**: Values exceeding 3σ represent less than 0.3% of normal variance (beyond 99.7% confidence interval). This level indicates significant deviation requiring immediate attention.

3. **T = 30 seconds**: Transient spikes are common during robot operation cycles. A 30-second sustained deviation filters out momentary fluctuations while capturing genuine degradation patterns.

In [ ]:
# Define thresholds based on residual analysis
print("=" * 70)
print("THRESHOLD CONFIGURATION")
print("=" * 70)

# Thresholds (multipliers of residual standard deviation)
MinC = 2.0   # Alert threshold: 2σ above regression
MaxC = 3.0   # Error threshold: 3σ above regression  
T = 30       # Sustained duration: 30 seconds

thresholds = AlertThresholds(MinC=MinC, MaxC=MaxC, T=T)
print(f"\nAlert Thresholds: {thresholds}")

print(f"\n--- THRESHOLD JUSTIFICATION ---")
print(f"""\nMinC = {MinC} (Alert Threshold)
  - Values exceeding {MinC}σ from regression line
  - Represents ~{100 - 95.45:.1f}% of normal variance (beyond 95% CI)
  - Indicates: Early signs of potential degradation
  - Action: Schedule inspection, monitor closely
""")

print(f"""MaxC = {MaxC} (Error Threshold)
  - Values exceeding {MaxC}σ from regression line
  - Represents ~{100 - 99.73:.2f}% of normal variance (beyond 99.7% CI)
  - Indicates: Significant anomaly, potential failure imminent
  - Action: Immediate attention required
""")

print(f"""T = {T} seconds (Sustained Duration)
  - Deviation must persist for {T} consecutive seconds
  - Filters out momentary spikes from normal operation
  - Based on typical robot operational cycle patterns
  - Action: Only alert on persistent issues
""")

# Show actual threshold values per axis (only for fitted models)
print("\nActual threshold values per axis (in normalized units):")
for axis_name in models.AXIS_NAMES:
    if models.models[axis_name].is_fitted:
        std = models.models[axis_name].residual_std
        print(f"  {axis_name}: ALERT >= {MinC * std:.4f}, ERROR >= {MaxC * std:.4f}")
    else:
        print(f"  {axis_name}: No model (no data)")

## Cell 7: Alert System Setup

In [ ]:
# Initialize alert system
alert_system = AlertSystem(models, thresholds)

print("Alert System initialized")
print(f"Thresholds: {alert_system.thresholds}")
print(f"Monitoring axes: {alert_system.AXIS_NAMES}")

# Create output directories
logs_dir = os.path.join(project_root, 'logs')
alerts_dir = os.path.join(project_root, 'alerts')
os.makedirs(logs_dir, exist_ok=True)
os.makedirs(alerts_dir, exist_ok=True)

print(f"Logs directory: {logs_dir}")
print(f"Alerts directory: {alerts_dir}")

## Cell 8: Streaming Test Simulation

Process the synthetic test data (`robots_combined_v2.csv`) which contains injected failure behavior.

In [ ]:
# Load test data (with injected failures)
test_csv_path = os.path.join(project_root, 'data', 'robots_combined_v2.csv')
print(f"Loading test data from: {test_csv_path}")

df_test = pd.read_csv(test_csv_path)
print(f"\nTest Data Shape: {df_test.shape}")
print(f"Columns: {list(df_test.columns)}")
print(f"Robots in test data: {df_test['robot'].unique()}")
print(f"\nTest data sample:")
df_test.head()

In [ ]:
# Process test data through alert system
# Using ALL test data for complete analysis
print(f"Processing all {len(df_test):,} test records...")

# Process streaming data
alerts_detected = alert_system.process_streaming_data(df_test, time_interval_seconds=1)

In [ ]:
# Display alert summary
print("=" * 70)
print("ALERT DETECTION RESULTS")
print("=" * 70)

print(f"\nTotal events logged: {len(alert_system.alert_log)}")

if alert_system.alert_log:
    # Count by type
    alert_count = len([e for e in alert_system.alert_log if e.event_type == 'ALERT'])
    error_count = len([e for e in alert_system.alert_log if e.event_type == 'ERROR'])
    
    print(f"  - ALERTs: {alert_count}")
    print(f"  - ERRORs: {error_count}")
    
    print("\nSummary by Axis:")
    summary = alert_system.get_alert_summary()
    print(summary.to_string())
    
    print("\nSample Events:")
    for event in alert_system.alert_log[:5]:
        print(f"  {event}")
else:
    print("\nNo alerts detected in test data.")
    print("Consider adjusting thresholds or using more test data.")

## Cell 9: Event Logging

In [ ]:
# Save alert log to CSV
log_path = os.path.join(logs_dir, 'alert_log.csv')
alert_system.save_log_to_csv(log_path)

# Display the saved log
if os.path.exists(log_path):
    df_log = pd.read_csv(log_path)
    print(f"\nLog file preview ({len(df_log)} records):")
    print(df_log.head(10).to_string())

## Cell 10: Alert Dashboard Visualization

Generate comprehensive dashboard showing all 12 axes with alert/error markers.

In [ ]:
# Generate comprehensive dashboard for all 12 axes
print("Generating alert dashboard for all 12 axes...")
dashboard_path = os.path.join(alerts_dir, 'alert_dashboard.png')
fig_dashboard = alert_system.generate_alert_dashboard(df_test, save_path=dashboard_path)
plt.show()

## Cell 11: Final Summary

In [ ]:
# Count fitted models
fitted_models = len([m for m in models.models.values() if m.is_fitted])

print("=" * 70)
print("PREDICTIVE MAINTENANCE ANALYSIS COMPLETE")
print("=" * 70)

print(f"""
SUMMARY:
--------
Training Data: {len(df_training):,} records from database
Test Data: {len(df_test):,} records processed
Models Trained: {fitted_models} / 12 axes (axes with data)

THRESHOLDS:
-----------
MinC (Alert): {thresholds.MinC}σ above regression
MaxC (Error): {thresholds.MaxC}σ above regression
T (Duration): {thresholds.T} seconds sustained

RESULTS:
--------
Total Events Detected: {len(alert_system.alert_log)}
  - ALERTs: {len([e for e in alert_system.alert_log if e.event_type == 'ALERT'])}
  - ERRORs: {len([e for e in alert_system.alert_log if e.event_type == 'ERROR'])}

OUTPUT FILES:
-------------
- Model Parameters: data/model_params.csv
- Alert Log: logs/alert_log.csv
- Regression Plot: alerts/regression_lines.png
- Residual Analysis: alerts/residual_histograms.png
- Residual Boxplots: alerts/residual_boxplots.png
- Alert Dashboard: alerts/alert_dashboard.png
""")

# Close database connection
conn.close()
print("\nDatabase connection closed.")
print("\n" + "=" * 70)